# Transfer Learning Hubble Image Similarity Processing

In [ ]:
import pickle
import os
import shutil

from tldist.fingerprint.processing import FingerprintCalculatorResnet
from tldist.fingerprint.processing import calculate as fingerprint_calculate
from tldist.similarity.similarity import calculate as similarity_calculate
from tldist.data import Data
from tldist.cutout.generators import FullImageCutoutGenerator
from tldist.database import get_database

db = get_database('blitzdb', 'basic_notebook_100.db')

## Load the pre-massaged data

The pickle file contains a list of dictionaries. Each dictionary represents an image and contains info about the file location, RA/DEC and meta data (observation information).

In [ ]:
# Load the data
processing_dict = pickle.load(open('../data/hubble_acs.pck', 'rb'))

## Save the Image Data

Each data element will be saved to a Data object in order to confirm input is correct and then save to a database for later retrival.

In [ ]:
data = []
for fileinfo in processing_dict[:100]:
    im = Data(location=fileinfo['location'], radec=fileinfo['radec'], meta=fileinfo['meta'])
    data.append(im)
    db.save('data', im.save())

## Create the Cutouts

Cutouts of the images will be made. The cutouts in this case represent a 224x224 cutout of the data and these cutouts will be stored in the database too. The cutouts could have other types of image processing applied to the underlying data.

In [ ]:
full_cutout = FullImageCutoutGenerator(output_size=(224, 224))

In [ ]:
cutouts = []
for datum in data:
    cutout = full_cutout.create_cutouts(datum)
    db.save('cutout', cutout.save())
    cutouts.append(cutout)

## Compute the Fingerprints

Compute the fingerprints from each of the cutouts based on, in this case, the Resnet pre-trained network.  The fingerprints are stored in the database too.  The Fingerprint calculator will calculate based on the Resnet, VGG16, VGG19, InceptionV3 or InceptionResnetV2 pre-trained networks.

In [ ]:
# This will take a minute as it is loading the pre-trained network
fresnet = FingerprintCalculatorResnet()
fc_save = fresnet.save()

In [ ]:
fingerprints = fingerprint_calculate(cutouts, fc_save)
[db.save('fingerprint', x.save()) for x in fingerprints]

## Compute the Similarity

Once the fingerprints are calculated, next the similarity is calculated. In this case the [tSNE algorithm](https://lvdmaaten.github.io/tsne/) is used which is a data reduction algorithm and enables determining similar cutouts given one cutout.

In [ ]:
similarity_tsne = similarity_calculate(fingerprints, 'tsne')
db.save('similarity', similarity_tsne.save())